In [7]:
import pandas as pd
import numpy as np
from sys import platform

In [8]:
if platform == 'win32':
    revenue = pd.read_excel('D:/OneDrive - Emory University/MSBA/Spring/ISOM 689 - Capstone/data/DLY_HTL_COMPSET_PERF_MASKED.xlsx')
    competitors = pd.read_excel('D:/OneDrive - Emory University/MSBA/Spring/ISOM 689 - Capstone/data/RATE_SHOP_COMPET_HTL_masked.xlsx')
elif platform == 'darwin':
    revenue = pd.read_excel('/Volumes/External/OneDrive - Emory University/MSBA/Spring/ISOM 689 - Capstone/data/DLY_HTL_COMPSET_PERF_MASKED.xlsx')
    competitors = pd.read_excel('/Volumes/External/OneDrive - Emory University/MSBA/Spring/ISOM 689 - Capstone/data/RATE_SHOP_COMPET_HTL_masked.xlsx')

In [9]:
revenue.shape

(384237, 17)

In [10]:
revenue.head()

,FAC_ID,RPT_DAY_DT_TY,htl_rms_avl,htl_rms_act,htl_rev_const_usd_act,comp_rms_avl,comp_rms_act,comp_rev_const_usd_act,htl_occ,htl_adr,htl_revpar,comp_occ,comp_adr,comp_revpar,mpi,ari,rgi
0,3,2017-01-01,122.0,23.0,2295.0,122.0,18.633441,1673.086817,0.188525,99.782609,18.811475,0.152733,89.789471,13.713826,1.234341,1.111295,1.371716
1,3,2017-01-02,122.0,23.0,2378.0,122.0,19.418006,1888.254019,0.188525,103.391304,19.491803,0.159164,97.242426,15.477492,1.184468,1.063232,1.259364
2,3,2017-01-03,122.0,30.0,3104.0,122.0,41.974277,4078.173633,0.245902,103.466667,25.442623,0.344051,97.158877,33.427653,0.714724,1.064922,0.761125
3,3,2017-01-04,122.0,46.0,4664.0,122.0,54.919614,5540.408360,0.377049,101.391304,38.229508,0.450161,100.882143,45.413183,0.837587,1.005047,0.841815
4,3,2017-01-05,122.0,45.0,4676.0,122.0,46.681672,4449.077170,0.368852,103.911111,38.327869,0.382637,95.306723,36.467846,0.963975,1.090281,1.051004


In [11]:
revenue = revenue.groupby(['RPT_DAY_DT_TY']).agg({
    'htl_rev_const_usd_act':'sum',
    'comp_rev_const_usd_act':'sum',
}).reset_index()

In [13]:
revenue['weekday'] = revenue['RPT_DAY_DT_TY'].dt.weekday.apply(lambda x: 1 if x>=5 and x<7 else 0)

In [14]:
for n in range(0,60):
    col_name = 'day' + str(n)
    revenue[col_name] = revenue['htl_rev_const_usd_act'].shift(365+n)

In [15]:
revenue[revenue.RPT_DAY_DT_TY=='2018-03-01']

,RPT_DAY_DT_TY,htl_rev_const_usd_act,comp_rev_const_usd_act,weekday,day0,day1,day2,day3,day4,day5,...,day50,day51,day52,day53,day54,day55,day56,day57,day58,day59
424,2018-03-01,4093360.0,3.877978e+06,0,4316241.0,4233594.0,3589104.0,2369283.0,3982147.0,3911686.0,...,3998577.0,3528333.0,2231125.0,2512777.0,2372526.0,2316549.0,2411719.0,2190306.0,1644727.0,1919278.0


In [16]:
train = revenue[(revenue['RPT_DAY_DT_TY'] < '2018-09-01') & (revenue['RPT_DAY_DT_TY'] >= '2018-03-01')]
test = revenue[revenue['RPT_DAY_DT_TY'] >= '2018-09-01']

In [17]:
train.tail()

,RPT_DAY_DT_TY,htl_rev_const_usd_act,comp_rev_const_usd_act,weekday,day0,day1,day2,day3,day4,day5,...,day50,day51,day52,day53,day54,day55,day56,day57,day58,day59
603,2018-08-27,3671148.0,3.553462e+06,0,2457140.0,3709073.0,3522285.0,3628440.0,4403684.0,4480180.0,...,4359916.0,4094881.0,3447903.0,3072110.0,2192627.0,2490848.0,2990652.0,4824523.0,4017469.0,3910728.0
604,2018-08-28,4208820.0,4.091091e+06,0,3550270.0,2457140.0,3709073.0,3522285.0,3628440.0,4403684.0,...,3011583.0,4359916.0,4094881.0,3447903.0,3072110.0,2192627.0,2490848.0,2990652.0,4824523.0,4017469.0
605,2018-08-29,4024523.0,3.881793e+06,0,3968156.0,3550270.0,2457140.0,3709073.0,3522285.0,3628440.0,...,4621548.0,3011583.0,4359916.0,4094881.0,3447903.0,3072110.0,2192627.0,2490848.0,2990652.0,4824523.0
606,2018-08-30,3398519.0,3.245674e+06,0,3826626.0,3968156.0,3550270.0,2457140.0,3709073.0,3522285.0,...,5309537.0,4621548.0,3011583.0,4359916.0,4094881.0,3447903.0,3072110.0,2192627.0,2490848.0,2990652.0
607,2018-08-31,4020019.0,3.921251e+06,0,3329539.0,3826626.0,3968156.0,3550270.0,2457140.0,3709073.0,...,5435038.0,5309537.0,4621548.0,3011583.0,4359916.0,4094881.0,3447903.0,3072110.0,2192627.0,2490848.0


In [18]:
train = train.drop('RPT_DAY_DT_TY', axis=1)

In [19]:
train = train.drop('comp_rev_const_usd_act', axis=1)

In [20]:
X_train = train.drop('htl_rev_const_usd_act', axis=1)
y_train = train['htl_rev_const_usd_act']

In [21]:
test = test.drop(columns=['RPT_DAY_DT_TY', 'comp_rev_const_usd_act'])
X_test = test.drop('htl_rev_const_usd_act', axis=1)
y_test = test['htl_rev_const_usd_act']

In [22]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

# Linear Regression

In [23]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [24]:
# R^2
lr.score(X_train, y_train)

0.8532661327419373

In [25]:
y_pred_lr = lr.predict(X_test)
mean_absolute_percentage_error(y_test, y_pred_lr)

0.1877118452562216

# Ridge

In [31]:
from sklearn.linear_model import Ridge

ridge = GridSearchCV(estimator = Ridge(),
                    param_grid=[{'alpha': [0.01, 0.1, 1]}],
                    cv=5,
                    n_jobs=-1)

ridge.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'alpha': [0.01, 0.1, 1]}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [32]:
y_pred_ridge = ridge.predict(X_test)
mean_absolute_percentage_error(y_test, y_pred_ridge)

0.1877542685298714

# Lasso

In [33]:
from sklearn.linear_model import Lasso

lasso = GridSearchCV(estimator=Lasso(),
                    param_grid={'alpha': [0.001, 0.01, 0.1, 1]},
                    cv=5,
                    n_jobs=-1)

lasso.fit(X_train, y_train)

/Users/fenghaogao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/fenghaogao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/fenghaogao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/fenghaogao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: 

GridSearchCV(cv=5, error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': [0.001, 0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [35]:
lasso.get_params()

{'cv': 5,
 'error_score': 'raise',
 'estimator': Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False),
 'estimator__alpha': 1.0,
 'estimator__copy_X': True,
 'estimator__fit_intercept': True,
 'estimator__max_iter': 1000,
 'estimator__normalize': False,
 'estimator__positive': False,
 'estimator__precompute': False,
 'estimator__random_state': None,
 'estimator__selection': 'cyclic',
 'estimator__tol': 0.0001,
 'estimator__warm_start': False,
 'fit_params': None,
 'iid': True,
 'n_jobs': -1,
 'param_grid': {'alpha': [0.001, 0.01, 0.1, 1]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': 'warn',
 'scoring': None,
 'verbose': 0}

In [34]:
y_pred_lasso = lasso.predict(X_test)
mean_absolute_percentage_error(y_test, y_pred_lasso)

0.187711846723981

# Regression Tree

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

gs_dt = GridSearchCV(estimator=DecisionTreeRegressor(random_state=41),
                  param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, None],
                              'criterion': ['mse', 'mae'],}],
                  cv=5,
                  n_jobs=-1)

gs_dt.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=41, splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, None], 'criterion': ['mse', 'mae']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
print(gs_dt.best_score_)
print(gs_dt.best_params_)
print(gs_dt.best_estimator_)

-0.02215879386906294
{'criterion': 'mae', 'max_depth': 2}
DecisionTreeRegressor(criterion='mae', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=41, splitter='best')


In [63]:
y_pred_rt = gs_dt.predict(X_test)
mean_absolute_percentage_error(np.array(y_test), y_pred_rt)

0.27822058175694075

# KNN

In [28]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

knn = GridSearchCV(estimator=KNeighborsRegressor(),
                  param_grid=[{'n_neighbors': [1, 3, 5, 7, 9],}],
                  cv=5,
                  n_jobs=-1)

knn.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'n_neighbors': [1, 3, 5, 7, 9]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [29]:
y_pred_knn = knn.predict(X_test)
mean_absolute_percentage_error(y_test, y_pred_knn)

0.2739353917653917

# Random Forest

In [36]:
from sklearn.ensemble import RandomForestRegressor

rf = GridSearchCV(estimator=RandomForestRegressor(),
                  param_grid={'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80],
                             'criterion': ['mse', 'mae'],
                             'max_depth': [5, 7, 9, 11, 13, 15],
                             },
                  cv=5,
                  n_jobs=-1)

rf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80], 'criterion': ['mse', 'mae'], 'max_depth': [5, 7, 9, 11, 13, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [38]:
y_pred_rf = rf.predict(X_test)
mean_absolute_percentage_error(y_test, y_pred_rf)

0.20226586764197846